In [1]:
from datetime import datetime
import sys
import time
import sqlite3
import logging
import csv
import random
import os
import shutil

In [2]:
qry0a = '''
    create temporary table small_listings as
    select currentPrice, datetime, date(datetime) "date", eventId, listingId, quantity
    from listings
;
'''

qry0b = '''
    create index i on small_listings (eventId, datetime, currentPrice)
;
'''

qry0c = '''
    create index i1 on small_listings (eventId, date)
;
'''

qry0_check = 'select count(distinct eventId) from small_listings'


qry1a = '''
    create temporary table low_price_table as
    select eventId, datetime
        , date(datetime, '-07 days') "week_back_date", datetime(datetime, '-001 minutes') "minute_back_date"
        , min(currentPrice) "low_price"
    from small_listings
    group by 1,2,3,4
;
'''

qry1b = '''
    create index i2 on low_price_table (eventId, datetime)
;
'''

qry2a = '''
    create temporary table event_datetime as
        select distinct eventId, datetime
        from low_price_table
;
'''

qry2b = '''
    create index i3 on event_datetime (eventId, datetime)
;
'''

qry2c = '''
    create temp table with_last_time as
    select lpt.eventId "eventId", lpt.datetime "datetime", lpt.low_price "low_price", max(ed.datetime) "last_datetime"
    from 
        low_price_table lpt 
        left join event_datetime ed
            on ed.eventId = lpt.eventId
            and ed.datetime between lpt.week_back_date and lpt.minute_back_date
    group by 1,2,3
;
'''

qry3a = '''
    create temporary table almost as
    select
        wlt.eventId "eventId"
        , e.eventDateLocal "eventDateLocal"
        , e.URL || '?sid=117035' "game_URL"
        , wlt.datetime "datetime"
        , date(wlt.datetime,'-001 days') "yest_date"
        , wlt.low_price "low_price"
        , lpt.low_price "last_price"
        , face.faceValue "faceValue"
        , wlt.low_price - face.faceValue "over_face"
        , (wlt.low_price - face.faceValue) / face.faceValue "over_face_pct"
        , wlt.low_price - min(lpt.low_price) "price_change"
        , (wlt.low_price - min(lpt.low_price)) / min(lpt.low_price) "price_change_pct"
        , (wlt.low_price - min(lpt.low_price)) / min(lpt.low_price) <= -.05 "price_flag"

    from
        with_last_time wlt
        left join low_price_table lpt
            on lpt.eventId = wlt.eventId
            and lpt.datetime = wlt.last_datetime

        left join events e using (eventId)
        left join facevalues face on e.eventDateLocal = face.eventDateLocal

    group by 1,2,3,4,5,6,7,8,9
;
'''

qry3b = '''
    create index i4 on almost (eventId, yest_date)
;
'''

qry4a = '''
    create temporary table yest_prices as
    select a.eventId "eventId", a.yest_date "yest_date", min(yest.currentPrice) "yest_price"
    from (select distinct eventId, yest_date from almost) a
        left join small_listings yest
            on yest.eventId = a.eventId
            and yest.date = a.yest_date
    group by 1,2
;
'''

qry4b = '''
    create temporary table output as
    select
        a.eventId "eventId"
        , a.eventDateLocal "eventDateLocal"
        , a.game_URL "game_URL"
        , a.datetime "datetime"
        , a.low_price "low_price"
        , a.last_price "last_price"
        , a.faceValue "faceValue"
        , a.over_face "over_face"
        , a.over_face_pct "over_face_pct"
        , a.price_change "price_change"
        , a.price_change_pct "price_change_pct"
        , a.price_flag "price_flag"


        , 'https://www.stubhub.com/buy/review?ticket_id=' || max(l.listingId) || '&quantity_selected=' || l.quantity || '&event_id=' || a.eventID "listing_URL"
        , max(l.listingId) "listingId"
        , l.quantity "quantity"

        , yest_price "yesterdays_best_price"
        , a.low_price - yest_price "yest_price_change"
        , a.low_price - yest_price <= -5 "yest_price_flag"

    from
        almost a
        left join small_listings l
            on l.eventId = a.eventId
            and l.datetime = a.datetime
            and l.currentPrice = a.low_price

        left join yest_prices yest
            on yest.eventId = a.eventId
            and yest.yest_date = a.yest_date

    group by 1,2,3,4,5,6,7,8,9,10,11,12,15
    order by 1,4 desc
;
'''

qry4c = '''
    select * 
    from output 
    where listingId not in (1190356184
                                , 1182095596
                                , 1189448408
                                , 1185291783
                                , 1188996138
                                , 1189714426
                                , 1173301639
                                , 1191082113
                                , 1186321666
                                , 1188996152
                                , 1194077594
                                , 1191332752
                                , 1183397283
                                , 1192388616
                                , 1191244655
                                , 1192807045
                                , 1194078039
                                , 1205118971
                                , 1193030368
                                , 1186948736
                                , 1190420808
                                , 1187297513
                                , 1191244655
                                , 1194078039
                                , 1186321666
                                , 1193739716
                                , 1192834490
                                , 1191250685
                                , 1202410775
                                , 1204728138
                                , 1206121146
                                , 1205117106
                                , 1185179420
                                )
     order by "datetime"
'''

qry4_check = 'select count(distinct eventId) from output'

qry5 = '''
    with different_dates as (
    select 
        eventId
        , eventDateLocal
        , faceValue
        , max(datetime) "most_recent_time"
    from output
    group by 1,2,3
    having max(datetime) > date('now', '-003 days')

    )

    , dd2 as (
        select 
            dd.*, max(o.datetime) "one_week_ago"
        from different_dates dd
            left join output o using (eventId)
        where 
            o.datetime <= datetime(dd.most_recent_time,'-007 days')
        group by 1,2,3,4
    )

    , dd3 as (
        select 
            dd2.*, max(o.datetime) "two_weeks_ago"
        from dd2
            left join output o using (eventId)
        where 
            o.datetime <= datetime(dd2.most_recent_time,'-014 days')
        group by 1,2,3,4,5
    )

    , dd4 as (
        select 
            dd3.*, max(o.datetime) "three_weeks_ago"
        from dd3
            left join output o using (eventId)
        where 
            o.datetime <= datetime(dd3.most_recent_time,'-021 days')
        group by 1,2,3,4,5,6
    )

    , dd5 as (
        select 
            dd4.*, max(o.datetime) "four_weeks_ago"
        from dd4
            left join output o using (eventId)
        where 
            o.datetime <= datetime(dd4.most_recent_time,'-028 days')
        group by 1,2,3,4,5,6,7
    )

    select 
        substr(dd.eventDateLocal,1,10) || ' ' || substr(dd.eventDateLocal,12,5) "gamedate"
        , dd.eventId "eventId"
        , dd.faceValue "face value"
        , date(dd.most_recent_time) "today"

        , cast(min(o0.low_price) as 'integer') "today's price"
        , cast(min(o1.low_price) as 'integer') "last week's price"
        , cast(min(o2.low_price) as 'integer') "price 2 weeks ago"
        , cast(min(o3.low_price) as 'integer') "price 3 weeks ago"
        , cast(min(o4.low_price) as 'integer') "price 4 weeks ago"

        , cast(round(100.0 * (min(o0.low_price) - min(o1.low_price))  / min(o1.low_price), 2) as text) || '%'  "change since 1 week ago"
        , cast(round(100.0 * (min(o0.low_price) - min(o2.low_price))  / min(o2.low_price), 2) as text) || '%'  "change since 1 week ago"
        , cast(round(100.0 * (min(o0.low_price) - min(o3.low_price))  / min(o3.low_price), 2) as text) || '%'  "change since 1 week ago"
        , cast(round(100.0 * (min(o0.low_price) - min(o4.low_price))  / min(o4.low_price), 2) as text) || '%'  "change since 1 week ago"

    from dd5 dd
        left join output o0
            on dd.eventId = o0.eventId
            and dd.most_recent_time = o0.datetime
        left join output o1 
            on dd.eventId = o1.eventId
            and dd.one_week_ago = o1.datetime
        left join output o2
            on dd.eventId = o2.eventId
            and dd.two_weeks_ago = o2.datetime
        left join output o3 
            on dd.eventId = o3.eventId
            and dd.three_weeks_ago = o3.datetime
        left join output o4 
            on dd.eventId = o4.eventId
            and dd.four_weeks_ago = o4.datetime
    group by 1,2,3,4
    order by 1
'''


In [11]:
file_path = 'C:/Users/Monstar/Python/Stubhub/Cubs'

good_file_name = '{}/stubhub_min.db'.format(file_path)
copy_file_name = '{}/stubhub_min_for_output.db'.format(file_path)
shutil.copy(good_file_name, copy_file_name)

db_min = sqlite3.connect(copy_file_name)
c = db_min.cursor()

In [12]:
c.execute(str(qry0a))

In [13]:
c.execute(str(qry0b))

In [14]:
c.execute(str(qry0c))

In [18]:
c.execute(str(qry0_check))

In [19]:
c.fetchone()

(38,)

In [20]:
c.execute(str(qry1a))

OperationalError: table low_price_table already exists

In [21]:
c.execute(str(qry1b))

In [22]:
c.execute(str(qry2a))


In [23]:
c.execute(str(qry2b))

In [24]:
c.execute(str(qry2c))

In [25]:
c.execute(str(qry3a))

In [26]:
c.execute(str(qry3b))

In [27]:
c.execute(str(qry4a))

In [28]:
c.execute(str(qry4b))

In [32]:
c.execute(str(qry4c))

In [30]:
c.execute(str(qry4_check))

In [49]:
c.execute('''            
            select 
                eventId
                , max(datetime) "most_recent_time"
            from small_listings
            group by 1
            having max(datetime) > date('now', '-013 days')
            order by 2
''')

In [50]:
c.fetchall()

[(9445290, '2016-08-09 09:52:21.327294'),
 (9445291, '2016-08-10 00:20:51.839845'),
 (9445308, '2016-08-11 01:28:03.927080'),
 (9445310, '2016-08-12 04:57:25.816376'),
 (9445312, '2016-08-14 12:52:12.407043'),
 (9445325, '2016-08-15 20:46:36.543895'),
 (9445300, '2016-08-15 23:20:19.660427'),
 (9445302, '2016-08-15 23:20:19.660427'),
 (9445313, '2016-08-15 23:20:19.660427'),
 (9445322, '2016-08-15 23:20:19.660427'),
 (9445323, '2016-08-15 23:20:19.660427'),
 (9445298, '2016-08-16 11:59:31.635851'),
 (9445299, '2016-08-17 10:32:13.268031'),
 (9445370, '2016-08-18 20:16:13.387009'),
 (9445369, '2016-08-18 23:56:18.126276'),
 (9445351, '2016-08-19 20:17:23.925841'),
 (9445352, '2016-08-19 21:34:27.967320'),
 (9445368, '2016-08-19 23:24:22.875528'),
 (9445301, '2016-08-20 01:14:26.434230'),
 (9445314, '2016-08-20 07:39:35.467991'),
 (9445349, '2016-08-20 07:39:35.467991')]

In [10]:
c.close()
db_min.close()